In [1]:
from TTS.api import TTS
from pydub import AudioSegment
import os
from tqdm import tqdm
from pathlib import Path as pt

/Users/aravindhnivas/Documents/GitHub/NLP-Projects-Notebooks/.venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


/Users/aravindhnivas/Documents/GitHub/NLP-Projects-Notebooks/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Using model: xtts


/Users/aravindhnivas/Documents/GitHub/NLP-Projects-Notebooks/.venv/lib/python3.9/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

In [3]:
def split_text(text, max_len=200):
    paragraphs = text.split("\n")
    chunks = []
    for paragraph in paragraphs:
        sentences = paragraph.split(".")
        chunk = ""
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) == 0:
                continue
            sentence += "."
            if len(chunk) + len(sentence) <= max_len:
                chunk += " " + sentence
            else:
                if len(sentence) > max_len:
                    for i in range(0, len(sentence), max_len):
                        sub_sentence = sentence[i:i+max_len]
                        if sub_sentence[-1] != ".":
                            sub_sentence += "."
                        chunks.append(sub_sentence)
                else:
                    chunks.append(chunk.strip())
                    chunk = sentence
        if chunk:
            chunks.append(chunk.strip())
    return chunks

def gen_speech(text, name='output', out_fmt="mp3", language="en", **kwargs):
    audio = AudioSegment.empty()
    outfilename = f"{name}_{language}"
    for ind, chunk in enumerate(tqdm(split_text(text))):
        file_path=f"./processed_audio/{outfilename}_chunk_{ind}.wav"
        tts.tts_to_file(text=chunk,
            file_path=file_path,
            language=language,
            **kwargs
        )
        
        audio_chunk = AudioSegment.from_file(file_path)
        audio += audio_chunk
    
    audio.export(f"./processed_audio/{outfilename}.{out_fmt}", format=out_fmt)
    
    # remove the chunk audio files
    # for ind in range(len(split_text(text))):
    #     file_path=f"{name}_{language}_chunk_{ind}.wav"
    #     os.remove(file_path)
    return audio

In [4]:
with open("./inputs/meluha_text_en.txt", "r") as f:
    text_en = f.read()

# gen_speech(text_en, name="meluha", language="en", speaker="Ana Florence")

In [5]:
with open("./inputs/meluha_text_hi.txt", "r") as f:
    text_hi = f.read()

# gen_speech(text_hi, name="meluha", language="hi", speaker="Ana Florence")
text_hi

'शिव ने नारंगी आकाश की ओर देखा। मानसरोवर के ऊपर मंडरा रहे बादल अस्त होते सूर्य को दिखाने के लिए अभी-अभी छंट चुके थे। जीवन का प्रतिभाशाली दाता एक बार फिर अपने दिन का आह्वान कर रहा था। शिव ने अपनी इक्कीस वर्ष की आयु में कुछ सूर्योदय देखे थे। लेकिन सूर्यास्त! उसने कोशिश की कि कभी भी सूर्यास्त न छूटे! किसी भी अन्य दिन, शिव ने सूर्य और हिमालय की शानदार पृष्ठभूमि के सामने विशाल झील का दृश्य देखा होगा, जहां तक \u200b\u200bआंख देख सकती थी। लेकिन आज नहीं।\n\nवह झील के ऊपर फैली संकरी कगार पर बैठ गया और अपना गठीला, मांसल शरीर बना लिया। उसकी त्वचा पर युद्ध के अनगिनत निशान पानी की झिलमिलाती परावर्तित रोशनी में चमक रहे थे। शिव को अपने बचपन के अल्हड़ दिन अच्छी तरह याद थे। उन्होंने झील की सतह से उछलकर आने वाले कंकड़ फेंकने की कला में महारत हासिल कर ली थी। उन्होंने अभी भी अपने समुदाय में सबसे अधिक बाउंस: सत्रह का रिकॉर्ड कायम रखा है।\n\nएक सामान्य दिन में, शिव अपने हँसमुख अतीत को याद करके मुस्कुरा देते थे जो वर्तमान के संताप से अभिभूत हो गया था। लेकिन आज वह बिना किसी खुशी के वापस अपने गांव की ओर मुड़ ग

In [14]:
# sound = AudioSegment.from_mp3("../NeonAI-Audio files/_Salman_Khan_Attitude_Dialogue_Ringtone_(by Fringster.com).mp3")
# audio_out = pt('./salmanKhan.wav')

audio_in = pt("../NeonAI-Audio files/Rishi_Sunak_s_first_speech_as_prime_minister (1).ogg")
print('audio_out: ', audio_in.exists())
sound = AudioSegment.from_ogg(audio_in)
# # audio_out = f'./{audio_in.stem}.wav'
audio_out = pt('./rishiSunak.wav')
print('audio_out: ', audio_out.exists())
if not audio_out.exists():
    sound.export(audio_out, format="wav")
    print('Exported')

audio_out:  True
audio_out:  True


In [16]:
audio = AudioSegment.from_wav(audio_out)
seconds = 25
ms = seconds * 1000
trimmed_audio = audio[:ms]
trimmed_filename = audio_out.parent / f"{audio_out.stem}_trimmed.wav"
print(trimmed_filename)
trimmed_audio.export(trimmed_filename, format="wav")

rishiSunak_trimmed.wav


<_io.BufferedRandom name='rishiSunak_trimmed.wav'>

In [17]:
# gen_speech(text_hi, name="meluha_cloned_modi_1_speed1", language="hi", speaker_wav="./modi_1.wav")
gen_speech(text_hi, name=f"{trimmed_filename}_generated_speech", language="hi", speaker_wav=trimmed_filename)

  0%|          | 0/6 [00:00<?, ?it/s]

 > Text splitted to sentences.
['शिव ने नारंगी आकाश की ओर देखा। मानसरोवर के ऊपर मंडरा रहे बादल अस्त होते सूर्य को दिखाने के लिए अभी-अभी छंट चुके थे। जीवन का प्रतिभाशाली दाता एक बार फिर अपने दिन का आह्वान कर रहा था। शिव ने अपनी इक्कीस.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
 17%|█▋        | 1/6 [00:46<03:51, 46.22s/it]

 > Processing time: 46.17774415016174
 > Real-time factor: 2.175756561145395
 > Text splitted to sentences.
['वर्ष की आयु में कुछ सूर्योदय देखे थे। लेकिन सूर्यास्त!', 'उसने कोशिश की कि कभी भी सूर्यास्त न छूटे!', 'किसी भी अन्य दिन, शिव ने सूर्य और हिमालय की शानदार पृष्ठभूमि के सामने विशाल झील का दृश्य देखा होगा, जह.']


 33%|███▎      | 2/6 [01:25<02:48, 42.02s/it]

...

 > Processing time: 35.72101807594299
 > Real-time factor: 2.129147875779981
 > Text splitted to sentences.
['तरह याद थे। उन्होंने झील की सतह से उछलकर आने वाले कंकड़ फेंकने की कला में महारत हासिल कर ली थी। उन्होंने अभी भी अपने समुदाय में सबसे अधिक बाउंस: सत्रह का रिकॉर्ड कायम रखा है।.']


 83%|████████▎ | 5/6 [02:50<00:33, 33.44s/it]

 > Processing time: 36.849278926849365
 > Real-time factor: 2.020245555200075
 > Text splitted to sentences.
['एक सामान्य दिन में, शिव अपने हँसमुख अतीत को याद करके मुस्कुरा देते थे जो वर्तमान के संताप से अभिभूत हो गया था। लेकिन आज वह बिना किसी खुशी के वापस अपने गांव की ओर मुड़ गया।.']


100%|██████████| 6/6 [03:19<00:00, 33.30s/it]

 > Processing time: 28.969120740890503
 > Real-time factor: 2.0705912307992183
